- rank	 : 해당일자의 박스오피스 순위를 출력합니다. 
- rankInten : 전일대비 순위의 증감분을 출력합니다.
- rankOldAndNew :	랭킹에 신규진입여부를 출력합니다.
- movieCd :	영화의 대표코드를 출력합니다.
- movieNm :	영화명(국문)을 출력합니다.
- openDt :	영화의 개봉일을 출력합니다.
- salesAmt :	해당일의 매출액을 출력합니다.
- salesShare :	해당일자 상영작의 매출총액 대비 해당 영화의 매출비율을 출력합니다.
- salesInten :	전일 대비 매출액 증감분을 출력합니다.
- salesChange :	전일 대비 매출액 증감 비율을 출력합니다.
- salesAcc :	누적매출액을 출력합니다.
- audiCnt :	해당일의 관객수를 출력합니다.
- audiInten :	전일 대비 관객수 증감분을 출력합니다.
- audiChange :	전일 대비 관객수 증감 비율을 출력합니다.
- audiAcc :	누적관객수를 출력합니다.
- scrnCnt :	해당일자에 상영한 스크린수를 출력합니다.
- showCnt :	해당일자에 상영된 횟수를 출력합니다.

In [183]:
pip install -U scikit-learn

In [184]:
import pandas as pd 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score 
import numpy as np
from sklearn.metrics import r2_score

In [185]:
import pickle

In [186]:
# API를 통해 가져온 데이터 불러오기 

movie = pd.read_csv('movie.csv')
movie.shape

(6390, 18)

In [187]:
print(movie.shape)
print(movie.dtypes)

(6390, 18)
Id                 int64
Rank               int64
RankInten          int64
RankOldAndNew     object
MovieCd            int64
MovieNm           object
OpenDt            object
SalesAmt           int64
SalesShare       float64
SalesInten         int64
SalesChange      float64
SalesAcc           int64
AudiCnt            int64
AudiInten          int64
AudiChange       float64
AudiAcc            int64
ScrnCnt            int64
ShowCnt            int64
dtype: object


#### **모델링에 적합한 Data 만들기**

In [188]:
# 총 영화 갯수 

movie_name = movie['MovieNm'].unique()
print(len(movie_name))

544


In [189]:
# 영화별 최대 스크린 수, 최대 상영횟수 구하기 

max = movie[['MovieNm', 'ScrnCnt', 'ShowCnt']].groupby('MovieNm').max().reset_index()
max

,MovieNm,ScrnCnt,ShowCnt
0,#살아있다,1882,9221
1,#아이엠히어,432,780
2,007 노 타임 투 다이,2178,6608
3,100% 울프: 푸들이 될 순 없어,370,468
4,1917,932,3844
...,...,...,...
539,호프,171,194
540,혼자 사는 사람들,159,208
541,화양연화,447,726
542,후쿠오카,93,139


In [190]:
highrank = movie[['MovieNm', 'Rank']].groupby('MovieNm').min().reset_index()
highrank.rename(columns= {'Rank' : 'HighRank'}, inplace=True)
highrank

,MovieNm,HighRank
0,#살아있다,1
1,#아이엠히어,2
2,007 노 타임 투 다이,1
3,100% 울프: 푸들이 될 순 없어,5
4,1917,1
...,...,...
539,호프,9
540,혼자 사는 사람들,6
541,화양연화,2
542,후쿠오카,6


In [191]:
# df로 구한 정보 합치기

df = max.merge(highrank, how='right', on = ['MovieNm'])

In [192]:
df.head()

,MovieNm,ScrnCnt,ShowCnt,HighRank
0,#살아있다,1882,9221,1
1,#아이엠히어,432,780,2
2,007 노 타임 투 다이,2178,6608,1
3,100% 울프: 푸들이 될 순 없어,370,468,5
4,1917,932,3844,1


In [193]:
# 머신러닝 데이터

ml_df = df.iloc[:, 1:]

In [194]:
ml_df.head()

,ScrnCnt,ShowCnt,HighRank
0,1882,9221,1
1,432,780,2
2,2178,6608,1
3,370,468,5
4,932,3844,1


#### **데이터 분석하기** 

In [195]:
ml_df.corr()

,ScrnCnt,ShowCnt,HighRank
ScrnCnt,1.000000,0.961858,-0.714714
ShowCnt,0.961858,1.000000,-0.644315
HighRank,-0.714714,-0.644315,1.000000


#### **예측 모델 만들기**

In [196]:
model = LinearRegression()

In [197]:
target = 'HighRank'
feature = ml_df.drop(columns=[target]).columns

In [198]:
X_train, X_test, y_train, y_test = train_test_split(ml_df[feature], ml_df[target], train_size=0.8, test_size=0.2)

In [199]:
model.fit(X_train, y_train)
y_pred_train = model.predict(X_train)
train_r2 = r2_score(y_train, y_pred_train)
print("Train r2 score : ", train_r2)

Train r2 score :  0.5252106556416136


In [200]:
y_pred_test = model.predict(X_test)
test_r2 = r2_score(y_test, y_pred_test)
print("Test r2 score : ", test_r2)

Test r2 score :  0.5513252744993289


In [201]:
score = cross_val_score(model, ml_df[feature], ml_df[target], scoring='r2')
score

array([0.50444433, 0.54498112, 0.52775462, 0.49727842, 0.55235006])

In [202]:
model.coef_

array([-0.00755794,  0.00068546])

In [203]:
print("평균 r2 score : ", np.sum(score)/np.size(score))

평균 r2 score :  0.525361710129622


In [207]:
model.predict([[1,0]])[0]

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:442: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  "X does not have valid feature names, but"


7.600240418229277

In [205]:
pickle.dump(model, open('movie.pkl', 'wb'))